In [1]:
from __future__ import division
import numpy as np
import scipy.io as scipy
import matplotlib.pyplot as plt
!pip install mne
import mne
from numpy import *
from numpy.fft import *
import scipy.signal as signal
from matplotlib.pyplot import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import sklearn
from mne.decoding import CSP

In [4]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], scipy.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, scipy.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, scipy.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [6]:
p2_all = loadmat('P2_AllLifts.mat')
print( "All Lifts loaded")



#these are the series we are looking at for subject 2. Everything except series 1 and 6
trials_included=[2, 3, 4, 5, 7, 8, 9] 

#corresponding row indices in P.AllLifts where each series starts. Index is off by one bc of how python indexes things
trialStartIdx = [28, 62, 96, 130, 192, 226, 260]

#data structure to hold all EEG trials 
eeg_grasp_closed = []
eeg_grasp_open = []
emg_grasp_closed = []
emg_grasp_open = []
series_counter = 0
trial_counter = 1
i = 0
for s in trials_included:
    # load all the series specified in trials_included
    series_filename = 'WS_P2_S{series}.mat'.format(series = s)
    p_series = loadmat(series_filename)
    print("Series data loaded")
    series_counter = series_counter + 1
    print("series_counter: ", series_counter)

    #identifying the starting row index in p.ALLLifts for the first trial of the current series
    tStart = trialStartIdx[i]
    i += 1
    trial_counter = 1
    #range for the for loop is 34 values bc there are 34 trials in each series
    for trial_num in range(tStart, tStart + 34):
        
        #retrieving the point in time when both fingers touched object
        #index values are one cloumn and one row before target value
        tBothDigitTouch = np.array(p2_all['P']['AllLifts'])[trial_num, 14]
        
        tBothReleased = np.array(p2_all['P']['AllLifts'])[trial_num, 22]
        
        #converting time points to row indices to be accessed in windowed struct
        #500 samples every second, so timepoint should be divided by 0.002
        idxCloseEnd = int(tBothDigitTouch // 0.002) + int(0.8//0.002)
        idxCloseStart = int(tBothDigitTouch // 0.002)- int(0.8//0.002)
        
        idxOpenEnd = int(tBothReleased // 0.002)+ int(0.8//0.002)
        idxOpenStart = int(tBothReleased // 0.002)- int(0.8//0.002)
        
        idxCloseEndEMG = int(tBothDigitTouch // 0.0002) + int(0.8//0.0002)
        idxCloseStartEMG = int(tBothDigitTouch // 0.0002) - int(0.8//0.0002)
        
        idxOpenEndEMG = int(tBothReleased // 0.0002) + int(0.8//0.0002)
        idxOpenStartEMG = int(tBothReleased // 0.0002) - int(0.8//0.0002)

        eeg_trial = np.array(p_series['ws']['win'][0]['eeg'])[idxCloseStart:idxCloseEnd].T 
        eeg_grasp_closed.append(eeg_trial)
        
        eeg_trial = np.array(p_series['ws']['win'][0]['eeg'])[idxOpenStart:idxOpenEnd].T 
        eeg_grasp_open.append(eeg_trial)
        
        emg_trial = np.array(p_series['ws']['win'][0]['emg'])[idxCloseStartEMG:idxCloseEndEMG].T 
        emg_grasp_closed.append(emg_trial)
        
        emg_trial = np.array(p_series['ws']['win'][0]['emg'])[idxOpenStartEMG:idxOpenEndEMG].T 
        emg_grasp_open.append(emg_trial)
        #print("trial counter: ", trial_counter)
        trial_counter = trial_counter +1
        
                                                            

All Lifts loaded
Series data loaded
series_counter:  1
Series data loaded
series_counter:  2
Series data loaded
series_counter:  3
Series data loaded
series_counter:  4
Series data loaded
series_counter:  5
Series data loaded
series_counter:  6
Series data loaded
series_counter:  7


In [7]:
series = [2, 3, 4, 5, 7, 8, 9]
eeg_close = {}
counter = 0
for i in range (0, len(series)):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(eeg_grasp_closed[s])
    eeg_close[series[i]] = new_list
print(eeg_close.keys())
print(eeg_close.values())



dict_keys([2, 3, 4, 5, 7, 8, 9])
dict_values([[array([[-485., -487., -491., ...,   55.,   71.,   52.],
       [-220., -203., -203., ...,  271.,  276.,  255.],
       [ 290.,  267.,  252., ...,  276.,  276.,  228.],
       ...,
       [ 509.,  557.,  531., ...,   45.,   24.,   48.],
       [ 458.,  512.,  454., ...,   86.,   73.,   79.],
       [ 314.,  382.,  335., ...,  183.,  158.,  139.]], dtype=float32), array([[-138., -146., -166., ..., -100.,  -74.,  -22.],
       [ 125.,  134.,   93., ...,  318.,  310.,  339.],
       [ 282.,  305.,  282., ...,  209.,  197.,  222.],
       ...,
       [ 226.,  150.,  103., ...,   13.,   -9.,   61.],
       [ 334.,  197.,  124., ...,   28.,   16.,   87.],
       [ 539.,  381.,  198., ...,  176.,   88.,  182.]], dtype=float32), array([[-522., -527., -556., ...,  -31.,  -20.,  -43.],
       [-253., -270., -264., ...,  273.,  262.,  228.],
       [ 197.,  163.,  153., ...,  152.,  169.,  216.],
       ...,
       [ 370.,  383.,  365., ...,   50.,   

In [8]:
series = [2, 3, 4, 5, 7, 8, 9]
eeg_open = {}
counter = 0
for i in range (0, len(series)):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(eeg_grasp_open[s])
    eeg_open[series[i]] = new_list
print(eeg_open.keys())
print(eeg_open.values())

dict_keys([2, 3, 4, 5, 7, 8, 9])
dict_values([[array([[-258., -286., -294., ...,  144.,  174.,  181.],
       [-199., -217., -212., ...,  -25.,  -13.,  -36.],
       [  40.,    2.,   15., ...,  354.,  356.,  351.],
       ...,
       [ 284.,  224.,  208., ...,  154.,  105.,  116.],
       [ 325.,  282.,  246., ...,  158.,   61.,   93.],
       [ 371.,  269.,  187., ...,  262.,   24.,   58.]], dtype=float32), array([[  288.,   307.,   323., ..., -1487., -1474., -1487.],
       [  307.,   311.,   301., ..., -1073., -1055., -1028.],
       [  300.,   306.,   303., ...,  -193.,  -225.,  -270.],
       ...,
       [ -164.,  -136.,  -161., ...,   585.,   638.,   600.],
       [ -107.,  -105.,  -126., ...,   639.,   704.,   672.],
       [  -66.,   -52.,    -2., ...,   605.,   755.,   687.]],
      dtype=float32), array([[ 320.,  305.,  295., ..., -713., -698., -714.],
       [ 272.,  279.,  292., ..., -500., -541., -551.],
       [ 266.,  242.,  261., ...,  -87.,  -44.,  -91.],
       ...,
 

In [9]:
series = [2, 3, 4, 5, 7, 8, 9]
emg_close = {}
counter = 0
for i in range (0, len(series)):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(emg_grasp_closed[s])
    emg_close[series[i]] = new_list
# print(emg_close.keys())
# print(emg_close.values())

In [10]:
series = [2, 3, 4, 5, 7, 8, 9]
emg_open = {}
counter = 0
for i in range (0, len(series)):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(emg_grasp_open[s])
    emg_open[series[i]] = new_list
# print(emg_open.keys())
# print(emg_open.values())

In [11]:
csp = CSP(n_components=2, reg=None, log=True, norm_trace=False)
labels_close_2 = [0 for x in range(len(eeg_close[2]))] 
labels_open_2 = [1 for x in range(len(eeg_open[2]))] 
labels = np.array(labels_close_2 + labels_open_2)
print(labels_open_2)
print(labels_close_2)
print(labels)
eeg_open_array = np.array(eeg_grasp_open)
eeg_open_array = np.array(eeg_grasp_closed)
# eeg_x = np.concatenate(eeg_grasp_open,eeg_grasp_closed)
# eeg_2 = np.concatenate(eeg_close_2_array,eeg_open_2_array)
# csp.fit_transform(eeg_2, labels_close_2)
# csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/Users/chaodanluo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


ValueError: could not broadcast input array from shape (32,800) into shape (32)

In [12]:
for num in range(len(eeg_close[2])):
    print(eeg_close[2][num].shape)




(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)


In [21]:
for num in range(len(eeg_open[2])):
    print(eeg_open[2][num].shape)
print(len(eeg_open[2]))


(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 532)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 682)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
34


In [15]:
for num in range(len(eeg_grasp_open)):
    
    print(eeg_grasp_open[num].shape)

(32, 800)
(32, 800)
(32, 789)
(32, 800)
(32, 744)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 728)
(32, 800)
(32, 800)
(32, 800)
(32, 781)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 678)
(32, 800)
(32, 800)
(32, 800)
(32, 118)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 592)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 741)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 709)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)


In [80]:
for num in range(len(eeg_grasp_closed)):
    print(eeg_grasp_closed[num].shape)

(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)
(32, 800)


In [82]:
csp = CSP(n_components=2, reg=None, log=True, norm_trace=False)
labels_close_2 = [0 for x in range(len(eeg_close[2]))] 
labels_open_2 = [1 for x in range(len(eeg_open[2]))] 
labels = np.array(labels_close_2 + labels_open_2)
print(labels_open_2)
print(labels_close_2)
print(labels)
# eeg_open_array = np.array(eeg_grasp_open)
# eeg_open_array = np.array(eeg_grasp_closed)
eeg_x = np.concatenate(eeg_grasp_open,eeg_grasp_closed)
# eeg_2 = np.concatenate(eeg_close_2_array,eeg_open_2_array)
# csp.fit_transform(eeg_2, labels_close_2)
# csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


TypeError: 'list' object cannot be interpreted as an integer

In [23]:
for num in range(len(eeg_open[2])):
    if eeg_open[2][num].shape[1] != 800:
        tBothDigitTouch = np.array(p2_all['P']['AllLifts'])[num, 14]
        tBothReleased = np.array(p2_all['P']['AllLifts'])[num, 22]
        idxCloseEnd = int(tBothDigitTouch // 0.002) + int(0.8//0.002)
        idxCloseStart = int(tBothDigitTouch // 0.002)- int(0.8//0.002)
        idxOpenEnd = int(tBothReleased // 0.002)+ int(0.8//0.002)
        idxOpenStart = int(tBothReleased // 0.002)- int(0.8//0.002)
        eeg_array = np.array(p_series['ws']['win'][0]['eeg'])[idxCloseStart:idxCloseEnd].T
        print("tBothDigitTouch is {}".format(tBothDigitTouch))
        print("tBothReleased is {}".format(tBothReleased))
        print("idxCloseEnd is {}".format(idxCloseEnd))
        print("idxCloseStart is {}".format(idxCloseStart))
        print("idxOpenEnd is {}".format(idxOpenEnd))
        print("idxOpenStart is {}".format(idxOpenStart))
        print("eeg_array is {}".format(eeg_array))
 


tBothDigitTouch is 5.194
tBothReleased is 10.614
idxCloseEnd is 2996
idxCloseStart is 2196
idxOpenEnd is 5707
idxOpenStart is 4907
eeg_array is [[-63. -53.   1. ... 185. 136. 146.]
 [146. 183. 223. ... 321. 260. 246.]
 [232. 252. 295. ... 309. 303. 310.]
 ...
 [141. 165. 163. ... -30.  14.  55.]
 [112. 171. 181. ...  72. 144. 108.]
 [122. 201. 263. ... 167. 398. 307.]]
tBothDigitTouch is 4.962
tBothReleased is 10.078
idxCloseEnd is 2880
idxCloseStart is 2080
idxOpenEnd is 5438
idxOpenStart is 4638
eeg_array is [[ -57.  -48.  -47. ...   29.   30.   45.]
 [ 304.  295.  301. ...  253.  263.  264.]
 [ 190.  191.  186. ...  124.  156.  137.]
 ...
 [  49.   53.  104. ... -136.  -72.  -14.]
 [  81.   84.  119. ...  -26.   40.   51.]
 [ 120.  144.  140. ...   76.  188.  233.]]


In [18]:
for num in range(len(eeg_grasp_open)):
    index_list = []
    if eeg_grasp_open[num].shape[1] != 800:
        index_list.append(num)
    print(index_list)

[]
[]
[2]
[]
[4]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[17]
[]
[]
[]
[21]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[32]
[]
[]
[]
[36]
[]
[]
[]
[]
[]
[]
[]
[]
[45]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[55]
[]
[]
[]
[]
[]
[]
[]
[63]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[108]
[]
[]
[]
[]
[]
[]
[]
[116]
[117]
[]
[119]
[]
[]
[]
[]
[]
[]
[]
[127]
[128]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[210]
[]
[]
[]
[]
[]
[]
[]
[218]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
